In [1]:
import pandas as pd
import numpy as np
import string
from scipy import stats 

In [2]:
cm_header = list(pd.read_csv(r"C:\Users\krred\Documents\Berkeley\w209\Final_P_Data\cm_header_file.csv").columns)
committee_master = pd.read_csv(r"C:\Users\krred\Documents\Berkeley\w209\Final_P_Data\cm_16.txt",sep = "|" ,names = cm_header)
committee_master.head()

,CMTE_ID,CMTE_NM,TRES_NM,CMTE_ST1,CMTE_ST2,CMTE_CITY,CMTE_ST,CMTE_ZIP,CMTE_DSGN,CMTE_TP,CMTE_PTY_AFFILIATION,CMTE_FILING_FREQ,ORG_TP,CONNECTED_ORG_NM,CAND_ID
0,C00000059,HALLMARK CARDS PAC,ERIN BROWER,2501 MCGEE,MD#288,KANSAS CITY,MO,64108,U,Q,UNK,M,C,NaN,NaN
1,C00000422,AMERICAN MEDICAL ASSOCIATION POLITICAL ACTION ...,"WALKER, KEVIN","25 MASSACHUSETTS AVE, NW",SUITE 600,WASHINGTON,DC,20001,B,Q,NaN,M,M,AMERICAN MEDICAL ASSOCIATION,NaN
2,C00000489,D R I V E POLITICAL FUND CHAPTER 886,TOM RITTER,3528 W RENO,NaN,OKLAHOMA CITY,OK,73107,U,N,NaN,Q,L,TEAMSTERS LOCAL UNION 886,NaN
3,C00000547,KANSAS MEDICAL SOCIETY POLITICAL ACTION COMMITTEE,"C. RICHARD BONEBRAKE, M.D.",623 SW 10TH AVE,NaN,TOPEKA,KS,66612,U,Q,UNK,Q,T,NaN,NaN
4,C00000638,INDIANA STATE MEDICAL ASSOCIATION POLITICAL AC...,"VIDYA KORA, M.D.","322 CANAL WALK, CANAL LEVEL",NaN,INDIANAPOLIS,IN,46202,U,Q,NaN,Q,M,NaN,NaN


In [3]:
cont_columns = list(pd.read_csv(r"C:\Users\krred\Documents\Berkeley\w209\Final_P_Data\indiv_header_file.csv").columns)

In [ ]:
zip_county = pd.read_csv(r"C:\Users\krred\Documents\Berkeley\w209\Final_Project\zip_county_cross.csv")
zip_county.head()

,County,STATE,lookup,STCOUNTYFP,CLASSFP,ZIP_CODE
0,Prince of Wales-Hyder,AK,princeofwaleshyderak,2198,H5,99927
1,Prince of Wales-Hyder,AK,princeofwaleshyderak,2198,H5,99926
2,Prince of Wales-Hyder,AK,princeofwaleshyderak,2198,H5,99919
3,Prince of Wales-Hyder,AK,princeofwaleshyderak,2198,H5,99918
4,Prince of Wales-Hyder,AK,princeofwaleshyderak,2198,H5,99921


In [ ]:
chunkappend = pd.DataFrame()
for chunk in pd.read_csv(r"C:\Users\krred\Documents\Berkeley\w209\Final_P_Data\itcont_2016.txt",sep = "|" ,names = cont_columns, chunksize=250000):
    chunk = pd.merge(chunk,committee_master, on = ['CMTE_ID'], how = 'inner')
    chunk = pd.merge(chunk, zip_county, on = "ZIP_CODE")
    by_fips = chunk.groupby(['STCOUNTYFP','CMTE_PTY_AFFILIATION','CAND_ID'])['TRANSACTION_AMT'].agg(['sum','count']).reset_index()
    chunkappend = chunkappend.append(by_fips)
    
chunkappend.head()

C:\Users\krred\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\krred\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (10,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\krred\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (10,15,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\krred\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (10,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=resu

In [ ]:
transfer = pd.read_csv(r"C:\Users\krred\Documents\Berkeley\w209\Final_Project\itpas2.txt",sep = "|", names = list(pd.read_csv(r"C:\Users\krred\Documents\Berkeley\w209\Final_Project\pas2_header_file.csv").columns))
transfer = pd.merge(transfer,committee_master, on = ['CMTE_ID'], how = 'inner', suffixes = ("","y"))
transfer[~transfer['CAND_ID'].isna()]
transfer = pd.merge(transfer, zip_county, on = "ZIP_CODE")
transfer = transfer.groupby(['STCOUNTYFP','CMTE_PTY_AFFILIATION','CAND_ID'])['TRANSACTION_AMT'].agg(['sum','count']).reset_index()
donations = chunkappend.append(transfer)

donations['STCOUNTYFP'] = donations['STCOUNTYFP'].astype(int)
donations['total_donations'] = donations.groupby(['STCOUNTYFP','CMTE_PTY_AFFILIATION','CAND_ID'])['sum'].transform(sum)
donations['donation_count'] = donations.groupby(['STCOUNTYFP','CMTE_PTY_AFFILIATION','CAND_ID'])['count'].transform(sum)
donations.drop_duplicates(['STCOUNTYFP','CMTE_PTY_AFFILIATION','CAND_ID'], inplace = True)
donations.rename(columns = {'STCOUNTYFP':"county_fips", "CAND_ID":"candidate_fec"}, inplace = True)
donations.drop(['sum','count'],axis = 1, inplace=True)
donations = donations[(donations['CMTE_PTY_AFFILIATION'] == 'DEM') | (donations['CMTE_PTY_AFFILIATION'] == 'REP')]
donations.to_csv(r'C:\Users\krred\Documents\Berkeley\w209\Final_Project\donate_by_fips.csv')
donations.head()

In [ ]:
check = pd.read_csv(r"C:\Users\krred\Documents\Berkeley\w209\Final_P_Data\2016-precinct-president.csv", encoding='latin1')

In [ ]:
votes = pd.DataFrame()
for level in ['house','local','president','senate','state']:
    temp = pd.read_csv(r"C:\Users\krred\Documents\Berkeley\w209\Final_P_Data\2016-precinct-{}.csv".format(level), encoding='latin1')
#     temp = temp[~temp['party'].isna()]
    temp['candidate_fec'] = np.where(temp['candidate_fec'].isna(),"",temp['candidate_fec'])
    if 'candidate_party' in temp.columns:
        print(level)
        temp['candidate_party'] = temp['candidate_party'].astype(str)
        temp['candidate_party'] = temp['candidate_party'].str.lower()
        temp['candidate_party'] = np.where(temp['candidate_party'].str.lower().str.strip() == 'democrat','democratic',temp['candidate_party'])
        temp['party'] = np.where((temp['party'].isna()) | (temp['party'].str.strip() == ""), temp['candidate_party'], temp['party'])
    
    temp = temp.groupby(['state_postal','county_fips','county_name','candidate','candidate_fec','party','office'])['votes'].sum().reset_index()
    temp['party'] = temp['party'].astype(str)
    temp['office'] = temp['office'].astype(str)
    
    temp = temp[(temp['party']== 'republican') | (temp['party']== 'democratic') | (temp['party']== 'democrat')]
    temp = temp[~(temp['office']== 'Straight Party')]
    votes = votes.append(temp)
votes = votes[~votes['county_fips'].isna()]
votes['party'] = np.where(votes['party']=='republican',"REP","DEM")
votes['county_fips'] = votes['county_fips'].astype(int)
votes.head()

In [ ]:
county_donations_tot = donations.groupby(['county_fips','CMTE_PTY_AFFILIATION'])['total_donations'].sum().reset_index()
county_donations_cnt = donations.groupby(['county_fips','CMTE_PTY_AFFILIATION'])['donation_count'].sum().reset_index()
county_donations = pd.merge(county_donations_tot,county_donations_cnt,on=['county_fips','CMTE_PTY_AFFILIATION'])
county_donations.rename(columns = {'CMTE_PTY_AFFILIATION':"party", "total_donations":'total_party_donations_county', "donation_count":"total_donation_count_county"}, inplace = True)
county_donations.head()

In [ ]:
income = pd.read_csv(r"C:\Users\krred\Documents\Berkeley\w209\Final_P_Data\income_pop_bycounty.csv",encoding = 'latin1')
income = income.drop_duplicates(['county_fips','Description'])
income = income.pivot(index='county_fips', columns='Description', values='2016').reset_index()
income.rename(columns={'Per capita personal income (dollars) 2/':"Per_Capita_Inc",'Personal income (thousands of dollars)':"Personal_Inc" ,'Population (persons) 1/':"Population"}, inplace = True)
income = income[~income['Personal_Inc'].isna()]
income['county_fips'] = income['county_fips'].astype(int)
income.drop(np.nan,axis=1,inplace =True)
income.head()

In [ ]:
votes_donate = pd.merge(votes, donations, on = ['county_fips','candidate_fec'], how = 'left')

votes_donate['total_donations'] = np.where(votes_donate['total_donations'].isna(),0, votes_donate['total_donations'])
votes_donate['total_votes'] = votes_donate.groupby(['office','party','state_postal'])['votes'].transform(sum)
votes_donate['total_donations'] = votes_donate.groupby(['office','party','state_postal'])['total_donations'].transform(sum)

votes_donate['votes_winner_amt'] = votes_donate.groupby(['office','party','state_postal'])['total_votes'].transform(max)
votes_donate['donate_winner_amt'] = votes_donate.groupby(['office','party','state_postal'])['total_donations'].transform(max)

votes_donate['votes_winner'] = np.where(votes_donate['votes_winner_amt'] == votes_donate['total_votes'],1,0)
votes_donate['donation_winner'] = np.where(votes_donate['donate_winner_amt'] == votes_donate['total_donations'],1,0)

votes_donate['total_county_votes'] = votes_donate.groupby(['office','party','state_postal','county_fips'])['votes'].transform(sum)
votes_donate['total_county_donations'] = votes_donate.groupby(['office','party','state_postal','county_fips'])['total_donations'].transform(sum)

votes_donate['county_votes_winner_amt'] = votes_donate.groupby(['office','party','state_postal','county_fips'])['total_county_votes'].transform(max)
votes_donate['county_donate_winner_amt'] = votes_donate.groupby(['office','party','state_postal','county_fips'])['total_county_donations'].transform(max)

votes_donate['county_votes_winner'] = np.where(votes_donate['county_votes_winner_amt'] == votes_donate['total_county_votes'],1,0)
votes_donate['county_donation_winner'] = np.where(votes_donate['total_county_donations'] == votes_donate['county_donate_winner_amt'],1,0)

votes_donate['total_party_votes_county'] = votes_donate.groupby(['party','county_fips'])['votes'].transform(max)
votes_donate = pd.merge(votes_donate, county_donations, on=['county_fips','party'], how='left')
votes_donate['total_party_donations_county'] = np.where(votes_donate['total_party_donations_county'].isna(),0, votes_donate['total_party_donations_county'])

votes_donate.drop(['county_votes_winner_amt','county_donate_winner_amt', 'donate_winner_amt', 'votes_winner_amt','CMTE_PTY_AFFILIATION'], axis = 1, inplace = True)
votes_donate['county_fips'] = votes_donate['county_fips'].astype(int)

votes_donate = pd.merge(income,votes_donate,on='county_fips', how='inner')

votes_donate['normalized_votes'] = votes_donate['votes'].astype(float) / votes_donate['Population'].astype(float)
votes_donate['normalized_donations'] = votes_donate['total_donations'].astype(float) / votes_donate['Per_Capita_Inc'].astype(float)

votes_donate.to_csv(r'C:\Users\krred\Documents\Berkeley\w209\Final_Project\county_race_vote_donate.csv')
votes_donate